# Indice CHF (Extensión)

## Introducción

En este trabajo, nos centramos en la creación de un índice esencial para comprender la climatología de huracanes en las regiones de Costa Rica y Nicaragua: el CHF (Coastal Hurricane Frequency). Esta métrica se basa en la técnica CHF, como se plantea en el artículo de Karthik Balaguru, "Increased U.S. coastal hurricane risk under climate change". Sin embargo, es importante destacar que hemos realizado modificaciones significativas en este índice. La versión que empleamos ha sido adaptada y ampliada por Luis Barboza, Shu Wei Chou y Mario Gomez para abordar tanto el impacto directo como el indirecto de los ciclones tropicales en estas regiones.

Este índice se encuentra en una fase preliminar y es objeto de constante refinamiento en el marco de esta investigación y tiene la siguiente forma:

$$
CHF_{m} = \textrm{Efecto Directo} + \textrm{Efecto Indirecto}
$$

$$
CHF_{m} = CHF + N_{i} \frac{l}{TimeStep*v} \frac{P}{\overline P_{r}} \frac{Viento}{\overline{Viento_{r}}}
$$

$$
CHF_{m} = n\frac{l}{TimeStep*v} + N_{i} \frac{l}{TimeStep*v} \frac{P}{\overline P_{r}} \frac{Viento}{\overline{Viento_{r}}}
$$
En donde, 
$$
n: \textrm{es el número de trayectorias de huracanes que pasan a través de un grado cuadrado}
$$ 
$$
N_i :\textrm{Cantidad de Huracanés que no pasan a traves de esa cuadrícula}
$$
$$
l: \textrm{la longitud de la cuadrícula}
$$
$$
v: \textrm{la velocidad de traslación}
$$
$$
TimeStep: \textrm{es el paso de tiempo}
$$
$$
P: \textrm{Precipitación en esa cuadrícula}
$$
$$
\overline P_{r}: \textrm{Precipitación en toda la región}
$$

## Metodología 

Se utilizara Python para el análisis de este cuaderno, en donde se utilizaron las siguientes librerias

### Librerias

-   **NumPy:** Libreria para realizar cálculos numéricos en Python. Es ampliamente utilizada para operaciones matemáticas y de manipulación de datos.

-   **Pandas:** Pandas es una biblioteca que proporciona estructuras de datos y herramientas para el análisis de datos. Es especialmente útil para la manipulación y exploración de datos tabulares.

-   **GeoPandas:** GeoPandas es una extensión de Pandas que agrega capacidades geoespaciales. Permite trabajar con datos geoespaciales, como GeoDataFrames, y realizar análisis espaciales.

-   **OS:** El módulo `os` proporciona funciones para interactuar con el sistema operativo, como la manipulación de rutas de archivos y directorios.

-   **Shapely:** Shapely es una biblioteca que se utiliza para el procesamiento y análisis de geometría plana. Permite trabajar con objetos geométricos como puntos, líneas y polígonos.

-   **pyogrio:** pyogrio es una biblioteca que se utiliza para leer y escribir datos geoespaciales en formatos como GeoJSON y Shapefile. Facilita la manipulación de datos geoespaciales.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os 
import shapely
from pyogrio import read_dataframe
from shapely.geometry import Polygon, Point

## Creación del Indicador CHF

En esta sección, llevaremos a cabo la implementación del CHF modificado, abordando tanto el impacto directo como el impacto indirecto, siguiendo la solicitud de los investigadores. Además, exploraremos el índice CHF, que engloba tanto el impacto directo como el indirecto de los ciclones tropicales en la región.

Es importante mencionar que, debido a ciertas limitaciones en la obtención de datos de la página de ERA5 (https://cds.climate.copernicus.eu/cdsapp#!/home), así como en la base del HURDAT, habrá años para los cuales no se dispone de información. Sin embargo, estas limitaciones, en su mayoría asociadas a huracanes más antiguos, no deberían tener un impacto significativo en nuestro análisis.

### Importación de Bases de Datos

En esta etapa, procederemos a importar diversas bases de datos esenciales para nuestro análisis. En primer lugar, incorporaremos la base de datos de Natural Earth (https://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/), que abarca alrededor de 258 países, proporcionando información geográfica clave.

Asimismo, importaremos la base de datos del HURDAT, la cual está condicionada a un intervalo de tiempo de 24 horas, y es fundamental para nuestro estudio de huracanes.

Finalmente, incorporaremos la base de datos de ERA5, que nos permitirá enriquecer nuestro análisis con información sobre la velocidad de los vientos y la precipitación, añadiendo una dimensión adicional a nuestra investigación.

In [23]:
paises = read_dataframe("Shapefiles_mundo/ne_110m_admin_0_countries.shp")
hurdat = pd.read_csv("CHF_Balagurù/Datos/hurdat2daily.csv")

Ahora que se importaron los paises, se busca trabajar con todo America Central más la zona de la penisula de México. Por lo cual se procede a manipular la base de datos, paises.

In [25]:
paises_interes = ["Costa Rica","Nicaragua","Belize","Panama","El Salvador","Mexico","Honduras","Guatemala"]

paises = paises[["SOVEREIGNT","geometry"]]
paises = paises[paises['SOVEREIGNT'].isin(paises_interes)]

In [27]:
paises

,SOVEREIGNT,geometry
27,Mexico,"POLYGON ((-117.12776 32.53534, -115.99135 32.6..."
33,Panama,"POLYGON ((-77.35336 8.67050, -77.47472 8.52429..."
34,Costa Rica,"POLYGON ((-82.54620 9.56613, -82.93289 9.47681..."
35,Nicaragua,"POLYGON ((-83.65561 10.93876, -83.89505 10.726..."
36,Honduras,"POLYGON ((-83.14722 14.99583, -83.48999 15.016..."
37,El Salvador,"POLYGON ((-89.35333 14.42413, -89.05851 14.340..."
38,Guatemala,"POLYGON ((-92.22775 14.53883, -92.20323 14.830..."
39,Belize,"POLYGON ((-89.14308 17.80832, -89.15091 17.955..."


In [29]:
hurdat[["year","month","day"]] = hurdat["dat"].str.split('-',expand=True) #Separamos la fecha

hurdat.drop(columns= 'dat', inplace= True) #Eliminamos la columna que contiene la fecha agrupada

hurdat = hurdat.astype({'year':'int'})

In [32]:
minx, miny, maxx, maxy = paises.total_bounds #Limites de los paises

paises = hurdat[((hurdat["Lon"]<= maxx)&
                            (hurdat["Lon"]>= minx))&
                            ((hurdat["Lat"]<= maxy)&
                            (hurdat["Lat"]>= miny))] #Delimitación de la zona

In [37]:
paises

,ATCF,Name,rows,Lat,Lon,MaxW,MinP,34NE,34SE,34SW,...,50SW,50NW,64NE,64SE,64SW,64NW,Rad,year,month,day
0,AL011851,UNNAMED,14,28.0,-94.8,80.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1851,06,24
1,AL011851,UNNAMED,14,28.1,-96.3,78.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1851,06,25
2,AL011851,UNNAMED,14,28.6,-98.5,55.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1851,06,26
3,AL011851,UNNAMED,14,30.2,-100.0,40.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1851,06,27
4,AL021851,UNNAMED,1,22.2,-97.6,80.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1851,07,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14742,AL152022,LISA,24,17.3,-87.4,70.0,989.0,54.0,54.0,26.0,...,4.0,22.0,11.0,6.0,0.0,8.0,13.0,2022,11,02
14743,AL152022,LISA,24,17.9,-91.2,33.8,1005.5,7.5,7.5,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18.8,2022,11,03
14744,AL152022,LISA,24,18.9,-94.7,26.2,1008.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,35.0,2022,11,04
14745,AL152022,LISA,24,19.7,-95.4,25.0,1008.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40.0,2022,11,05


In [ ]:
precip = pd.read_csv('')